**Enter GDTOT Link as well as your GDTOT Crypt! If you don't know how to get Crypt then <a href="https://www.youtube.com/watch?v=EfZ29CotRSU">Learn Here</a>**

In [3]:
#@markdown <center><img src='https://gdtot.one/assets/img/gdtot.png' height="100">
#@title <b><center>Enter GDTOT-LINK Below</center></b>
print("Importing Files!")
import re
import base64
import requests
from urllib.parse import urlparse, parse_qs

URL = "https://new.gdtot.art/file/4026767219" #@param {type:"string"}
crypt = "L2xpc2x6TmtxL082OWZQZjlTUHQ4b0hhSE5mTW00ZTZrVkhVQk1kazh3MD0%3D" #@param {type:"string"}
print("You have Entered:")
print("URL:")
print(URL)
print("Crypt:")
print(crypt)
# ==========================================
print("Bypassing Link...")
def parse_info(res):
    title = re.findall(">(.*?)<\/h5>", res.text)[0]
    info = re.findall('<td\salign="right">(.*?)<\/td>', res.text)
    parsed_info = {
        'error': True,
        'message': 'Link Invalid.',
        'title': title,
        'size': info[0],
        'date': info[1]
    }
    return parsed_info

# ==========================================

def gdtot_dl(url):
    client = requests.Session()
    client.cookies.update({ 'crypt': crypt })
    res = client.get(url)

    info = parse_info(res)
    info['src_url'] = url

    res = client.get(f"https://new.gdtot.top/dld?id={url.split('/')[-1]}")
    
    try:
        url = re.findall('URL=(.*?)"', res.text)[0]
    except:
        info['message'] = 'The requested URL could not be retrieved.',
        return info

    params = parse_qs(urlparse(url).query)
    
    if 'msgx' in params:
        info['message'] = params['msgx'][0]
    
    if 'gd' not in params or not params['gd'] or params['gd'][0] == 'false':
        return info
    
    try:
        decoded_id = base64.b64decode(str(params['gd'][0])).decode('utf-8')
        gdrive_url = f'https://drive.google.com/open?id={decoded_id}'
        info['message'] = 'Success.'
    except:
        info['error'] = True
        return info

    info['gdrive_link'] = gdrive_url
    
    return info
    
# ==========================================

info = gdtot_dl(URL)

print(info)
print("Bypassed Successfully!")

Importing Files!


ModuleNotFoundError: No module named 'requests'

**Enter AppDrive or DriveApp and as well as there Account Details (Required for Login Required Links only)**

In [8]:
#@markdown <center><img src='https://i.imgur.com/sVViVY2.png' height="100">
#@title <b><center>Enter App Drive or Drive App Link Below</center></b>
print("Importing Files!")
import re
import requests
from lxml import etree
from urllib.parse import urlparse


# Website User Account (NOT GOOGLE ACCOUNT) ----
url = "https://appdrive.in/file/0511bac8c1c262ffce5d" #@param {type:"string"}
email = "fluidmechanicsfl@gmail.com" #@param {type:"string"}
passwd = "fluidmec@70" #@param {type:"string"}
print("You have Entered:")
print("Link:")
print(url)
print("Email:")
print(email)
print("Password:")
print(passwd)
# Destination config ----
SHARED_DRIVE_ID = '' # team drive ID (optional) (for MyDrive, keep this field empty)
FOLDER_ID = '' # drive folder ID (optional)

'''
NOTE: 
 - Auto-detection for non-login urls, and indicated via 'link_type' (direct/login) in output.
SUPPORTED DOMAINS:
 - appdrive.in
 - driveapp.in
 
'''
print("Bypassing Link...")
# ===================================================================

def account_login(client, url, email, password):
    data = {
        'email': email,
        'password': password
    }
    client.post(f'https://{urlparse(url).netloc}/login', data=data)

def update_account(client, url, shared_drive_id, folder_id):
    data = {
        'root_drive': shared_drive_id,
        'folder': folder_id
    }
    client.post(f'https://{urlparse(url).netloc}/account', data=data)

def gen_payload(data, boundary=f'{"-"*6}_'):
    data_string = ''
    for item in data:
        data_string += f'{boundary}\r\n'
        data_string += f'Content-Disposition: form-data; name="{item}"\r\n\r\n{data[item]}\r\n'
    data_string += f'{boundary}--\r\n'
    return data_string

def parse_info(data):
    info = re.findall('>(.*?)<\/li>', data)
    info_parsed = {}
    for item in info:
        kv = [s.strip() for s in item.split(':', maxsplit = 1)]
        info_parsed[kv[0].lower()] = kv[1]
    return info_parsed

# ===================================================================

def appdrive_dl(url):
    client = requests.Session()
    client.headers.update({
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"
    })

    account_login(client, url, email, passwd)
    update_account(client, url, SHARED_DRIVE_ID, FOLDER_ID)

    res = client.get(url)
    key = re.findall('"key",\s+"(.*?)"', res.text)[0]

    ddl_btn = etree.HTML(res.content).xpath("//button[@id='drc']")

    info_parsed = parse_info(res.text)
    info_parsed['error'] = False
    info_parsed['link_type'] = 'login' # direct/login
    
    headers = {
        "Content-Type": f"multipart/form-data; boundary={'-'*4}_",
    }
    
    data = {
        'type': 1,
        'key': key,
        'action': 'original'
    }
    
    if len(ddl_btn):
        info_parsed['link_type'] = 'direct'
        data['action'] = 'direct'
    
    while data['type'] <= 3:
        try:
            response = client.post(url, data=gen_payload(data), headers=headers).json()
            break
        except: data['type'] += 1
        
    if 'url' in response:
        info_parsed['gdrive_link'] = response['url']
    elif 'error' in response and response['error']:
        info_parsed['error'] = True
        info_parsed['error_message'] = response['message']
    else:
        info_parsed['error'] = True
        info_parsed['error_message'] = 'Something went wrong :('
    
    if info_parsed['error']: return info_parsed
    
    if urlparse(url).netloc == 'driveapp.in' and not info_parsed['error']:
        res = client.get(info_parsed['gdrive_link'])
        drive_link = etree.HTML(res.content).xpath("//a[contains(@class,'btn')]/@href")[0]
        info_parsed['gdrive_link'] = drive_link
        
    info_parsed['src_url'] = url
    
    return info_parsed

# ===================================================================

print(appdrive_dl(url))


Importing Files!
You have Entered:
Link:
https://appdrive.in/file/0511bac8c1c262ffce5d
Email:
fluidmechanicsfl@gmail.com
Password:
fluidmec@70
Bypassing Link...
{'name': 'Cyber.Vaar.S01.E10..1080p.VOOT.WEB-DL.DD5.1.H.264-Cybertron.mkv', 'format': 'mkv', 'size': '1.3GB', 'error': True, 'link_type': 'login', 'error_message': "The user's Drive storage quota has been exceeded."}


**Enter Sharer.pw Link, XSRF_TOKEN and laravel_session cookies! If you don't know how to get then then watch this <a href="https://www.youtube.com/watch?v=EfZ29CotRSU">Video</a> (for GDTOT) and do the same for Sharer.pw**

In [6]:
#@markdown <center><img src='https://sharer.pw/images/logo2.png' height="30">
#@title <b><center>Enter Sharer.pw Link Below</center></b>
print("Importing Files!")
import re
import requests
from lxml import etree

url = "https://sharer.pw/file/WlfijM24lfGz" #@param {type:"string"}

XSRF_TOKEN = "" #@param {type:"string"}
laravel_session = "" #@param {type:"string"}
print("You have Entered:")
print("URL:")
print(url)
print("XSRF_TOKEN:")
print(XSRF_TOKEN)
print("laravel_session")
print(laravel_session)
print("Bypassing Link...")
'''
404: Exception Handling Not Found :(
NOTE:
DO NOT use the logout button on website. Instead, clear the site cookies manually to log out.
If you use logout from website, cookies will become invalid.
'''

# ===================================================================

def parse_info(res):
    f = re.findall(">(.*?)<\/td>", res.text)
    info_parsed = {}
    for i in range(0, len(f), 3):
        info_parsed[f[i].lower().replace(' ', '_')] = f[i+2]
    return info_parsed

def sharer_pw_dl(url, forced_login=False):
    client = requests.Session()
    
    client.cookies.update({
        "XSRF-TOKEN": XSRF_TOKEN,
        "laravel_session": laravel_session
    })
    
    res = client.get(url)
    token = re.findall("_token\s=\s'(.*?)'", res.text, re.DOTALL)[0]
    
    ddl_btn = etree.HTML(res.content).xpath("//button[@id='btndirect']")

    info_parsed = parse_info(res)
    info_parsed['error'] = True
    info_parsed['src_url'] = url
    info_parsed['link_type'] = 'login' # direct/login
    info_parsed['forced_login'] = forced_login
    
    headers = {
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'x-requested-with': 'XMLHttpRequest'
    }
    
    data = {
        '_token': token
    }
    
    if len(ddl_btn):
        info_parsed['link_type'] = 'direct'
    if not forced_login:
        data['nl'] = 1
    
    try: 
        res = client.post(url+'/dl', headers=headers, data=data).json()
    except:
        return info_parsed
    
    if 'url' in res and res['url']:
        info_parsed['error'] = False
        info_parsed['gdrive_link'] = res['url']
    
    if len(ddl_btn) and not forced_login and not 'url' in info_parsed:
        # retry download via login
        return sharer_pw_dl(url, forced_login=True)
    
    return info_parsed

# ===================================================================

out = sharer_pw_dl(url)
print(out)

Importing Files!
You have Entered:
URL:
https://sharer.pw/file/WlfijM24lfGz
XSRF_TOKEN:

laravel_session

Bypassing Link...
{'file_name': 'Forensic 2022 1080p ZEE5 WEB-DL DD+5.1 H264-PHDM.mkv', 'type': 'video/x-matroska', 'size': '2.08GB', 'added_on': '2022-06-23', 'error': True, 'src_url': 'https://sharer.pw/file/WlfijM24lfGz', 'link_type': 'login', 'forced_login': False}


**Enter GP-LINK Below!**

In [11]:
#@markdown <center><img src='https://gplinks.in/img/gp-logo.png' height="100">
#@title <b><center>Enter GP-LINK Below</center></b>
print("Downloading Cloud-Scraper...")
!pip install cloudscraper
print("Setting Up!")
print("Performing Check...")
import time
import requests
import cloudscraper
from bs4 import BeautifulSoup
from urllib.parse import urlparse
print("Everything Looks Good! Lets Continue.")

url = "https://gplinks.co/WSu5rXI" #@param {type:"string"} 
print("Entered Link:")
print(url)
print("Checking Link...")
print("Checking Done!")
# ==============================================
print("Bypassing the Link...")
def gplinks_bypass(url: str):
    client = cloudscraper.create_scraper(allow_brotli=False)
    p = urlparse(url)
    final_url = f'{p.scheme}://{p.netloc}/links/go'
    
    res = client.head(url)
    header_loc = res.headers['location']
    param = header_loc.split('postid=')[-1]
    req_url = f'{p.scheme}://{p.netloc}/{param}'
    
    p = urlparse(header_loc)
    ref_url = f'{p.scheme}://{p.netloc}/'


    h = {'referer': ref_url}
    res = client.get(req_url, headers=h, allow_redirects=False)
    bs4 = BeautifulSoup(res.content, 'html.parser')
    inputs = bs4.find_all('input')
    time.sleep(10) # !important
    data = { input.get('name'): input.get('value') for input in inputs }
    
    h = {
        'content-type': 'application/x-www-form-urlencoded',
        'x-requested-with': 'XMLHttpRequest'
    }
    time.sleep(10)
    res = client.post(final_url, headers=h, data=data)
    try:
        return res.json()['url'].replace('/','/')
    except: return 'Something went wrong :('

# ==============================================

print(gplinks_bypass(url))
print("Confirming Link...")
print("Successfully Bypassed!")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Setting Up!
Performing Check...
Everything Looks Good! Lets Continue.
Entered Link:
https://gplinks.co/WSu5rXI
Checking Link...
Checking Done!
Bypassing the Link...
Something went wrong :(
Confirming Link...
Successfully Bypassed!


**Enter Adfly Link Below!**

In [ ]:
#@markdown <center><img src='https://1.bp.blogspot.com/-xUfQ3IdGsqw/V0CFRI9k84I/AAAAAAAAAjw/DLLzv392DVIgzG4jryRkd3EaulWtQ7nUwCLcB/s1600/earnwithadfly800x533.png' height="100">
#@title <b><center>Enter Adfly Link Below</center></b>
print("Importing Files!")
import re
import requests
from base64 import b64decode
from urllib.parse import unquote

# ==========================================

# adfly short url
URL = "" #@param {type:"string"}
print("You Have Entered:")
print(URL)
print("Checking Link...")
'''
404: Complete exception handling not found :(
'''
print("Bypassing Link...")
# ==========================================

def decrypt_url(code):
    a, b = '', ''
    for i in range(0, len(code)):
        if i % 2 == 0: a += code[i]
        else: b = code[i] + b

    key = list(a + b)
    i = 0

    while i < len(key):
        if key[i].isdigit():
            for j in range(i+1,len(key)):
                if key[j].isdigit():
                    u = int(key[i]) ^ int(key[j])
                    if u < 10: key[i] = str(u)
                    i = j					
                    break
        i+=1
    
    key = ''.join(key)
    decrypted = b64decode(key)[16:-16]

    return decrypted.decode('utf-8')

# ==========================================

def adfly_bypass(url):
    res = requests.get(url).text
    
    out = {'error': False, 'src_url': url}
    
    try:
        ysmm = re.findall("ysmm\s+=\s+['|\"](.*?)['|\"]", res)[0]
    except:
        out['error'] = True
        return out
        
    url = decrypt_url(ysmm)

    if re.search(r'go\.php\?u\=', url):
        url = b64decode(re.sub(r'(.*?)u=', '', url)).decode()
    elif '&dest=' in url:
        url = unquote(re.sub(r'(.*?)dest=', '', url))
    
    out['bypassed_url'] = url
    
    return out

# ==========================================

out = adfly_bypass(URL)

print(out)


**Enter DropLink Below!**

In [12]:
#@markdown <center><img src='https://i.imgur.com/1rWL0jS.png' height="50">
#@title <b><center>Enter Drop Link Below</center></b>
print("Importing Files!")
import re
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse


# droplink url
url = "https://droplink.co/KixiIauu" #@param {type:"string"}
print("You Have Entered:")
print(url)
print("Checking Link!")
# ==============================================
print("Bypassing Link...")
def droplink_bypass(url):
    client = requests.Session()
    res = client.get(url)

    ref = re.findall("action[ ]{0,}=[ ]{0,}['|\"](.*?)['|\"]", res.text)[0]

    h = {'referer': ref}
    res = client.get(url, headers=h)

    bs4 = BeautifulSoup(res.content, 'lxml')
    inputs = bs4.find_all('input')
    data = { input.get('name'): input.get('value') for input in inputs }

    h = {
        'content-type': 'application/x-www-form-urlencoded',
        'x-requested-with': 'XMLHttpRequest'
    }
    p = urlparse(url)
    final_url = f'{p.scheme}://{p.netloc}/links/go'

    time.sleep(3.1)
    res = client.post(final_url, data=data, headers=h).json()

    return res

# ==============================================

print(droplink_bypass(url))

Importing Files!
You Have Entered:
https://droplink.co/KixiIauu
Checking Link!
Bypassing Link...
{'status': 'success', 'message': 'Go without Earn because Adblock', 'url': 'https://fileclub.cyou/view/LQGyFHhdAT'}


In [ ]:
#@markdown <center><img src='https://linkvertise.com/assets/img/final-Logo.png' height="50">
#@title <b><center>Enter Linkvertise-LINK Below</center></b>
import re
import time
import json
import base64
import requests

# -------------------------------------------

def lv_bypass(url):
    client = requests.Session()
    
    headers = {
        "User-Agent": "AppleTV6,2/11.1",
        "Content-Type": "application/json",
        "Accept": "application/json",
    }
    
    client.headers.update(headers)
    
    url = url.replace("%3D", " ").replace("&o=sharing", "").replace("?o=sharing", "").replace("dynamic?r=", "dynamic/?r=")
    
    id_name = re.search(r"\/\d+\/[^\/]+", url)
    
    if not id_name: return None
    
    paths = [
        "/captcha", 
        "/countdown_impression?trafficOrigin=network", 
        "/todo_impression?mobile=true&trafficOrigin=network"
    ]
    
    for path in paths:
        url = f"https://publisher.linkvertise.com/api/v1/redirect/link{id_name[0]}{path}"
        response = client.get(url).json()
        if response["success"]: break
    
    data = client.get(f"https://publisher.linkvertise.com/api/v1/redirect/link/static{id_name[0]}").json()

    out = {
        'timestamp':int(str(time.time_ns())[0:13]),
        'random':"6548307", 
        'link_id':data["data"]["link"]["id"]
    }
    
    options = {
        'serial': base64.b64encode(json.dumps(out).encode()).decode()
    }
    
    data = client.get("https://publisher.linkvertise.com/api/v1/account").json()
    user_token = data["user_token"] if "user_token" in data.keys() else None
    
    url_submit = f"https://publisher.linkvertise.com/api/v1/redirect/link{id_name[0]}/target?X-Linkvertise-UT={user_token}"
    
    data = client.post(url_submit, json=options).json()
    
    return data["data"]["target"]

# -------------------------------------------

# Add URL
url = "https://domain.tld/XXXXXX/XXX" #@param {type:"string"}
bypassed = lv_bypass(url)

print(bypassed)